# Extraction task

- ABSA quad task

In [1]:
from datasets import load_dataset

ds = load_dataset("neudm/absa-quad")

README.md:   0%|          | 0.00/2.83k [00:00<?, ?B/s]

Generation%2Ftrain.jsonl:   0%|          | 0.00/2.00M [00:00<?, ?B/s]

Generation%2Fdev.jsonl:   0%|          | 0.00/503k [00:00<?, ?B/s]

Generation%2Ftest.jsonl:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 2098
    })
    validation: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 525
    })
    test: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 1081
    })
})

In [3]:
ds["train"][0]

{'task_type': 'generation',
 'dataset': 'absa-quad',
 'input': '["The wait here is long for dim sum , but if you do n\'t like sharing tables or if the typical raucous dim sum atmosphere is not your gig , this is a sleek ( for Chinatown ) alternative ."]',
 'output': "[['wait', 'service general', 'negative', 'long'], ['atmosphere', 'ambience general', 'negative', 'raucous'], ['NULL', 'restaurant miscellaneous', 'negative', 'sleek']]",
 'situation': 'none',
 'label': '',
 'extra': '',
 'instruction': 'Task: Extracting aspect terms and their corresponding aspect categories, sentiment polarities, and opinion words. Input: A sentence. Output: A list of 4-tuples, where each tuple contains the extracted aspect term, its aspect category, sentiment polarity, and opinion words (if any). Supplement: "Null" means that there is no occurrence in the sentence. Example:  Sentence: "The ambience was so fun , and the prices were great , on top of the fact that the food was really tasty"  Output: [[\'amb

In [6]:
import ast

tmp = ast.literal_eval(ds["train"][0]["output"])

In [7]:
print(tmp, type(tmp))

[['wait', 'service general', 'negative', 'long'], ['atmosphere', 'ambience general', 'negative', 'raucous'], ['NULL', 'restaurant miscellaneous', 'negative', 'sleek']] <class 'list'>


In [10]:

all_categories = []
all_sentiments = []

for sample in ds["train"]:
    review_text = sample["input"][2:-2]
    #print(review_text)
    tmp = ast.literal_eval(sample["output"])
    for _, cat, sent, _ in tmp:
        all_categories.append(cat)
        all_sentiments.append(sent)

In [12]:
all_categories = sorted(set(all_categories))
print(all_categories)

['ambience general', 'drinks prices', 'drinks quality', 'drinks style_options', 'food general', 'food prices', 'food quality', 'food style_options', 'location general', 'restaurant general', 'restaurant miscellaneous', 'restaurant prices', 'service general']


In [13]:
all_sentiments = sorted(set(all_sentiments))
print(all_sentiments)

['negative', 'neutral', 'positive']


['ambience general', 'drinks prices', 'drinks quality', 'drinks style_options', 'food general', 'food prices', 'food quality', 'food style_options', 'location general', 'restaurant general', 'restaurant miscellaneous', 'restaurant prices', 'service general']

In [30]:
SYSTEM_PROMPT = "You are an expert at aspect-based sentiment analysis."

In [32]:
grpo_train_dataset = []

for sample in ds["train"]:
    review_text = sample["input"][2:-2]

    user_prompt = """Below is a restaurant review.

Identify all the targets of opinions expressed by the reviewer, then classify the sentiment towards that target, then classify the category that best describes that opinion.

If the target of the opinion is only referred to implicitly, use the string NULL to represent the target.

Use only the sentiment labels and the aspect category labels given here:

## Sentiment labels

- negative
- neutral
- positive

## Aspect category labels

- ambience general
- drinks prices
- drinks quality
- drinks style_options
- food general
- food prices
- food quality
- food style_options
- location general
- restaurant general
- restaurant miscellaneous
- restaurant prices
- service general

Return your answer as a valid JSON object as shown in the example below:

## JSON schema

input review : The pizza was really cheap, but the restaurant was in a bad neighborhood. Overall I would recommend.

output answer : [ { "target" : "pizza", "sentiment" : "positive", "category" : "food prices"}, { "target" : "restaurant", "sentiment" : "negative", "category" : "location general"}, { "target" : "NULL", "sentiment" : "positive", "category" : "restaurant general"} ]

## Review

"""
    user_prompt += review_text
    tmp = ast.literal_eval(sample["output"])
    answer = []
    for target, cat, sent, _ in tmp:
        curr = {"target": target, "sentiment": sent, "category":cat}
        answer.append(curr)
    
    gold_answer = json.dumps(answer)
    if user_prompt is not None and gold_answer is not None:
        curr_datapoint = { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': user_prompt}
        ],
        'answer': gold_answer}
        grpo_train_dataset.append(curr_datapoint)

In [33]:
grpo_train_dataset[0]

{'prompt': [{'role': 'system',
   'content': 'You are an expert at aspect-based sentiment analysis.'},
  {'role': 'user',
   'content': 'Below is a restaurant review.\n\nIdentify all the targets of opinions expressed by the reviewer, then classify the sentiment towards that target, then classify the category that best describes that opinion.\n\nIf the target of the opinion is only referred to implicitly, use the string NULL to represent the target.\n\nUse only the sentiment labels and the aspect category labels given here:\n\n## Sentiment labels\n\n- negative\n- neutral\n- positive\n\n## Aspect category labels\n\n- ambience general\n- drinks prices\n- drinks quality\n- drinks style_options\n- food general\n- food prices\n- food quality\n- food style_options\n- location general\n- restaurant general\n- restaurant miscellaneous\n- restaurant prices\n- service general\n\nReturn your answer as a valid JSON object as shown in the example below:\n\n## JSON schema\n\ninput review : The pizza 

In [34]:
import re

# Reward functions

- todo: here only do the JSON itself and labels are in ALLOWED
- have not checked the SUBSTRING EXTRACTIVE BEHAVIOR

In [45]:
def schema_and_extractive(prompts, completions, answer, **kwargs) -> list[float]:
    print("----------")
    print("PROMPTS ===>", prompts)
    print("---")
    print("COMPLETIONS ===>", completions)
    print("----")
    print("ANSWER ===>", answer)
    responses = [completion[0]['content'] for completion in completions]
    scores = []
    for resp in responses:
        score = 0
        try:
            model_resp = json.loads(resp)
            score += 1.5
            for d in model_resp:
                if "target" in d.keys():
                    score += 0.2
                if "sentiment" in d.keys():
                    score += 0.2
                    if d["sentiment"] in all_sentiments:
                        score += 0.3
                    else:
                        score -= 0.1
                if "category" in d.keys():
                    score += 0.2
                    if d["sentiment"] in all_categories:
                        score += 0.3
                    else:
                        score -= 0.1

        except Exception as e:
            score = 0
        scores.append(score)
    return scores

In [36]:
import pandas as pd
from datasets import Dataset

# convert to HF dataset
df = pd.DataFrame(grpo_train_dataset)
train_data = Dataset.from_pandas(df, split="train")

In [37]:
train_data

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 2098
})

In [38]:
train_data[0]

{'prompt': [{'content': 'You are an expert at aspect-based sentiment analysis.',
   'role': 'system'},
  {'content': 'Below is a restaurant review.\n\nIdentify all the targets of opinions expressed by the reviewer, then classify the sentiment towards that target, then classify the category that best describes that opinion.\n\nIf the target of the opinion is only referred to implicitly, use the string NULL to represent the target.\n\nUse only the sentiment labels and the aspect category labels given here:\n\n## Sentiment labels\n\n- negative\n- neutral\n- positive\n\n## Aspect category labels\n\n- ambience general\n- drinks prices\n- drinks quality\n- drinks style_options\n- food general\n- food prices\n- food quality\n- food style_options\n- location general\n- restaurant general\n- restaurant miscellaneous\n- restaurant prices\n- service general\n\nReturn your answer as a valid JSON object as shown in the example below:\n\n## JSON schema\n\ninput review : The pizza was really cheap, b

# Now training and Unsloth stuff

In [39]:
from huggingface_hub import notebook_login

notebook_login()

In [40]:
!pip install unsloth vllm

# feb 2025
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

!pip install triton==3.1.0
!pip install -U pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 26.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 85.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━

In [42]:
from unsloth import FastLanguageModel, PatchFastRL

# feb 2025
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [43]:
RANDOM_SEED = 1729


In [44]:
from unsloth import is_bfloat16_supported

import torch

In [46]:
# 12hr training it seems in Kaggle
#MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
#MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct" # want to see if can use a very small model for the task in any case

In [47]:
max_seq_length = 1024
#max_seq_length = 2048 # 12hr training with 7B Qwen
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank, # todo: check if should do 2x or just 1x rank
    use_gradient_checkpointing = "unsloth",
    random_state = RANDOM_SEED,
)

INFO 03-03 22:42:52 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.66%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 4.9 GB. Also swap space = 5 GB.
WARNING 03-03 22:42:55 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-03 22:43:07 config.py:549] This model suppor

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [48]:
!pwd

/kaggle/working


In [49]:
!mkdir outputs

In [50]:
# look again to make sure format is ok
train_data

Dataset({
    features: ['prompt', 'answer'],
    num_rows: 2098
})

In [51]:
train_data[0]

{'prompt': [{'content': 'You are an expert at aspect-based sentiment analysis.',
   'role': 'system'},
  {'content': 'Below is a restaurant review.\n\nIdentify all the targets of opinions expressed by the reviewer, then classify the sentiment towards that target, then classify the category that best describes that opinion.\n\nIf the target of the opinion is only referred to implicitly, use the string NULL to represent the target.\n\nUse only the sentiment labels and the aspect category labels given here:\n\n## Sentiment labels\n\n- negative\n- neutral\n- positive\n\n## Aspect category labels\n\n- ambience general\n- drinks prices\n- drinks quality\n- drinks style_options\n- food general\n- food prices\n- food quality\n- food style_options\n- location general\n- restaurant general\n- restaurant miscellaneous\n- restaurant prices\n- service general\n\nReturn your answer as a valid JSON object as shown in the example below:\n\n## JSON schema\n\ninput review : The pizza was really cheap, b

In [52]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 512,
    max_completion_length = 256,
    #num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

In [53]:
import time

In [54]:
start_time = time.time()

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        schema_and_extractive,
    ],
    args = training_args,
    train_dataset = train_data,
)

train_results = trainer.train()

end_time = time.time()

print("TIME -> ", end_time-start_time)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,098 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 250
 "-____-"     Number of trainable parameters = 119,734,272


----------
PROMPTS ===> [[{'content': 'You are an expert at aspect-based sentiment analysis.', 'role': 'system'}, {'content': 'Below is a restaurant review.\n\nIdentify all the targets of opinions expressed by the reviewer, then classify the sentiment towards that target, then classify the category that best describes that opinion.\n\nIf the target of the opinion is only referred to implicitly, use the string NULL to represent the target.\n\nUse only the sentiment labels and the aspect category labels given here:\n\n## Sentiment labels\n\n- negative\n- neutral\n- positive\n\n## Aspect category labels\n\n- ambience general\n- drinks prices\n- drinks quality\n- drinks style_options\n- food general\n- food prices\n- food quality\n- food style_options\n- location general\n- restaurant general\n- restaurant miscellaneous\n- restaurant prices\n- service general\n\nReturn your answer as a valid JSON object as shown in the example below:\n\n## JSON schema\n\ninput review : The pizza was really

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / schema_and_extractive
1,-0.000000,2.220312,0.983752,45.921875,0.000000,2.220312
2,0.000000,1.576562,0.957608,54.250000,0.000000,1.576562
3,0.000000,2.132812,0.944673,47.625000,0.000023,2.132812
4,0.000000,2.175000,0.896595,51.578125,0.000028,2.175000
5,0.000000,1.867188,1.219710,42.015625,0.000021,1.867188
6,0.000000,2.028125,1.338575,51.734375,0.000066,2.028125
7,0.000000,2.329687,0.898418,50.312500,0.000327,2.329687
8,0.000000,2.432813,1.022546,58.375000,0.000482,2.432813
9,0.000100,2.284375,0.892637,49.546875,0.002500,2.284375
10,0.000300,2.728125,0.500028,49.937500,0.006687,2.728125


----------
PROMPTS ===> [[{'content': 'You are an expert at aspect-based sentiment analysis.', 'role': 'system'}, {'content': 'Below is a restaurant review.\n\nIdentify all the targets of opinions expressed by the reviewer, then classify the sentiment towards that target, then classify the category that best describes that opinion.\n\nIf the target of the opinion is only referred to implicitly, use the string NULL to represent the target.\n\nUse only the sentiment labels and the aspect category labels given here:\n\n## Sentiment labels\n\n- negative\n- neutral\n- positive\n\n## Aspect category labels\n\n- ambience general\n- drinks prices\n- drinks quality\n- drinks style_options\n- food general\n- food prices\n- food quality\n- food style_options\n- location general\n- restaurant general\n- restaurant miscellaneous\n- restaurant prices\n- service general\n\nReturn your answer as a valid JSON object as shown in the example below:\n\n## JSON schema\n\ninput review : The pizza was really

KeyboardInterrupt: 

# Interrupted

- seems to work, will try on larger GPU for speed

In [ ]:
SAVE_NAME = MODEL_NAME.split('/')[-1]

# save LoRA adapters here
model.push_to_hub_merged(f"benjaminzwhite/{SAVE_NAME}_Quad-GRPO_LoRA-adapters", tokenizer, save_method = "lora", token = "")